In [1]:
%matplotlib inline
import io
import urllib
import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as LA
import epopt as ep

data = "http://epopt.s3.amazonaws.com/mnist.npz"
mnist = np.load(io.BytesIO(urllib.urlopen(data).read()))

In [5]:
mnist["X"].shape

(60000, 784)

In [6]:
mnist["Y"].shape

(60000, 1)

In [7]:
mnist["Xtest"].shape

(10000, 784)

In [8]:
mnist["Ytest"].shape

(10000, 1)

In [9]:
# Problem data
X = mnist["X"] / 255.   # raw pixel data scaled to [0, 1]
y = mnist["Y"].ravel()  # labels {0, ..., 9}
Xtest = mnist["Xtest"] / 255.
ytest = mnist["Ytest"].ravel()

In [11]:
ytest.shape

(10000,)

In [12]:
# Parameters
m, n = X.shape
k = 10
Theta = cp.Variable(n, k)
lam = 1

In [13]:
# Form problem with CVXPY and solve with Epsilon
f = ep.multiclass_hinge_loss(Theta, X, y) + lam*cp.sum_squares(Theta)
prob = cp.Problem(cp.Minimize(f))
ep.solve(prob, verbose=True)

Epsilon 0.3.2
Compiled prox-affine form:
objective:
  add(
    affine(dense(A)*var(x)),
    non_negative(var(y)),
    affine(kron(dense(B), dense(C))*diag(D)*var(Z)),
    sum_square(var(W)))

constraints:
  zero(add(add(kron(transpose(dense(B)), scalar(1.00))*var(x), scalar(-1.00)*add(kron(scalar(1.00), dense(K))*var(W), dense(e)*1.00, const(F))), scalar(-1.00)*var(y)))
  zero(add(var(Z), scalar(-1.00)*var(W)))
Epsilon compile time: 0.8580 seconds

constraints, m = 607840, variables, n = 675680
iter=0 residuals primal=1.29e+05 [1.29e+03] dual=2.24e+02 [1.29e+03]
iter=40 residuals primal=9.62e+00 [1.02e+01] dual=2.54e+01 [1.29e+03]
Epsilon solve time: 28.4502 seconds


('optimal', 15982.950647475356)

In [16]:
# Get solution and compute train/test error
def error(x, y):
    return 1 - np.sum(x == y) / float(len(x))

Theta0 = np.array(Theta.value)
print "Train error:", error(np.argmax(X.dot(Theta0), axis=1), y)
print "Test error:", error(np.argmax(Xtest.dot(Theta0), axis=1), ytest)

Train error: 0.0853166666667
Test error: 0.0891


In [17]:
def median_dist(X):
    """Compute the approximate median distance by sampling pairs."""
    k = 1<<20  # 1M random points
    i = np.random.randint(0, X.shape[0], k)
    j = np.random.randint(0, X.shape[0], k)
    return np.sqrt(np.median(np.sum((X[i,:] - X[j,:])**2, axis=1)))
    
def pca(X, dim):
    """Perform centered PCA."""
    X = X - X.mean(axis=0)
    return LA.eigh(X.T.dot(X))[1][:,-dim:]

# PCA and median trick
np.random.seed(0)
V = pca(mnist["X"], 50)
X = mnist["X"].dot(V)
sigma = median_dist(X)

# Random features
n = 4000
W = np.random.randn(X.shape[1], n) / sigma
b = np.random.uniform(0, 2*np.pi, n)
X = np.cos(X.dot(W) + b)
Xtest = np.cos(mnist["Xtest"].dot(V).dot(W) + b)

In [18]:
# Parameters
m, n = X.shape
k = 10
Theta = cp.Variable(n, k)
lam = 10

# Form problem with CVXPY and solve with Epsilon
f = ep.multiclass_hinge_loss(Theta, X, y) + lam*cp.sum_squares(Theta)
prob = cp.Problem(cp.Minimize(f))
ep.solve(prob, verbose=True)

# Get solution and compute train/test error
Theta0 = np.array(Theta.value)
print "Train error:", error(np.argmax(X.dot(Theta0), axis=1), y)
print "Test error:", error(np.argmax(Xtest.dot(Theta0), axis=1), ytest)

Epsilon 0.3.2
Compiled prox-affine form:
objective:
  add(
    affine(dense(A)*var(x)),
    non_negative(var(y)),
    affine(kron(dense(B), dense(C))*diag(D)*var(Z)),
    sum_square(var(W)))

constraints:
  zero(add(add(kron(transpose(dense(B)), scalar(1.00))*var(x), scalar(-1.00)*add(kron(scalar(1.00), dense(K))*var(W), dense(e)*1.00, const(F))), scalar(-1.00)*var(y)))
  zero(add(var(Z), scalar(-1.00)*var(W)))
Epsilon compile time: 7.2581 seconds

constraints, m = 640000, variables, n = 740000
iter=0 residuals primal=7.12e+05 [7.12e+03] dual=2.71e+02 [7.12e+03]
iter=30 residuals primal=6.94e+00 [7.43e+00] dual=1.70e+01 [7.12e+03]
Epsilon solve time: 246.6345 seconds
Train error: 0.00501666666667
Test error: 0.0157
